In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data preparation

In [33]:
def normalize_col(col):
    avg=col.mean()
    std=col.std()
    return col.map(lambda x: (x-avg)/std)
def turn_categorical(df,cols):
    for col in cols:
        df=pd.concat([df,pd.get_dummies(df[col],prefix=col,prefix_sep='_')],axis=1).drop(col,axis=1)
    return df

In [59]:
data=pd.read_csv('data/ecommerce_data.csv')
data.head()

,is_mobile,n_products_viewed,visit_duration,is_returning_visitor,time_of_day,user_action
0,1,0,0.657510,0,3,0
1,1,1,0.568571,0,2,1
2,1,0,0.042246,1,1,0
3,1,1,1.659793,1,1,2
4,0,1,2.014745,1,1,2


In [61]:
data[['n_products_viewed','visit_duration']]=data[['n_products_viewed','visit_duration']].apply(normalize_col)
data.head()

,is_mobile,n_products_viewed,visit_duration,is_returning_visitor,time_of_day,user_action
0,1,-0.816161,-0.407869,0,3,0
1,1,0.139531,-0.498929,0,2,1
2,1,-0.816161,-1.037804,1,1,0
3,1,0.139531,0.618313,1,1,2
4,0,0.139531,0.981728,1,1,2


In [62]:
data=turn_categorical(data,['time_of_day'])
data.head()

,is_mobile,n_products_viewed,visit_duration,is_returning_visitor,user_action,time_of_day_0,time_of_day_1,time_of_day_2,time_of_day_3
0,1,-0.816161,-0.407869,0,0,0,0,0,1
1,1,0.139531,-0.498929,0,1,0,0,1,0
2,1,-0.816161,-1.037804,1,0,0,1,0,0
3,1,0.139531,0.618313,1,2,0,1,0,0
4,0,0.139531,0.981728,1,2,0,1,0,0


In [63]:
data=data[data['user_action']<=1]
data.head()

,is_mobile,n_products_viewed,visit_duration,is_returning_visitor,user_action,time_of_day_0,time_of_day_1,time_of_day_2,time_of_day_3
0,1,-0.816161,-0.407869,0,0,0,0,0,1
1,1,0.139531,-0.498929,0,1,0,0,1,0
2,1,-0.816161,-1.037804,1,0,0,1,0,0
6,0,-0.816161,0.393614,1,0,0,1,0,0
7,1,-0.816161,-1.044956,0,0,0,0,0,1


In [65]:
X=data.drop('user_action',axis=1).as_matrix()
Y=data['user_action'].as_matrix()
X.shape, Y.shape

((398, 8), (398,))

## Forward propagation

In [113]:
def process(X,W,b):
    return X.dot(W)+b
def sigmoid(v):
    return 1/(1+np.exp(-v))
def softmax(v):
    expv=np.exp(v)
    if v.shape[0]>1:
        return expv/expv.sum(axis=1,keepdims=True)
    else:
        return expv/expv.sum(axis=1)
def forward(X,W,b,func=sigmoid,softmax_output=True):
    v=X
    for l in range(len(W)-1):
        v=func(process(v,W[l],b[l]))
    if softmax_output:
        return softmax(process(v,W[-1],b[-1]))
    else:
        return process(v,W[-1],b[-1])
def classification_rate(Y,P):
    tot=Y.shape[0]
    return 1.0*sum([Y[i]==P[i] for i in range(tot)])/tot

In [91]:
num_nodes=[X.shape[1],5,len(set(Y))]

In [92]:
W=[]
b=[]
for i in range(len(num_nodes)-1):
    W.append(np.random.randn(num_nodes[i],num_nodes[i+1]))
    b.append(np.random.randn(num_nodes[i+1]))

In [93]:
preds=forward(X,W,b)
preds[:5]

array([[ 0.68372059,  0.31627941],
       [ 0.75017692,  0.24982308],
       [ 0.61748304,  0.38251696],
       [ 0.35598518,  0.64401482],
       [ 0.69161426,  0.30838574]])

In [94]:
pred_labels=np.argmax(preds,axis=1)
pred_labels[:5]

array([0, 0, 0, 1, 0])

In [95]:
classification_rate(Y,pred_labels)

0.3869346733668342